In [ ]:
#Import required libraries

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import progressbar

from datetime import datetime
from scipy import signal
from pydicom import dcmread
from pydicom.waveforms import multiplex_array

#Set options
np.set_printoptions(threshold = 500)

%matplotlib inline

In [ ]:
# Define class for reading ECGs from DICOM files.

class ECGDICOMReader:
    """ Extract voltage data from a ECG DICOM file
        Author: Philip Croon, p.croon@amsterdamumc.nl
        for questions feel free to email.    
        
        --Update 2023-09-19  Added information on Manufacturer, Accession number, Software and the Median Waveforms by Stephan van der Zwaard
    """

    def __init__(self, augmentLeads=False, resample_500=True):
        """ 
        Initialize class. If resample_500 is True ECGs with sampling frequency that are not 500 will be resampled to 500. 
        If AugmentLeads = True and the augmented leads are not available, they are calculated. 
        """
        self.augmentLeads = augmentLeads
        self.resample_500 = resample_500
        print("Initialization succesfull")

    def __call__(self, path, verbose=False):
        
        try:
            with open(path, 'rb') as DICOM:
                
                # Open DICOM file
                self.ECG                   = dcmread(DICOM)
                
                # Add DICOM information by relevant tags
                # General information
                
                # Check if key DICOM tags are present
                if set(['SOPInstanceUID','SeriesInstanceUID','StudyInstanceUID','PatientID','AccessionNumber','StudyDate','StudyTime']).issubset(self.ECG.dir()) == True:
                    self.SOPinstanceUID        = self.ECG.SOPInstanceUID
                    self.SERIESinstanceUID     = self.ECG.SeriesInstanceUID
                    self.STUDYinstanceUID      = self.ECG.StudyInstanceUID
                    self.PatientID             = self.ECG.PatientID
                    self.StudyDate             = self.ECG.StudyDate
                    self.StudyTime             = self.ECG.StudyTime
                    self.AccessionNumber       = self.ECG.AccessionNumber
                else:
                    if (verbose==True): 
                        print('Essential DICOM tags are missing: inspect DICOM file')
                    return(pd.DataFrame({'filetype': [self.ECG.data_element('SOPClassUID').repval], 'error': 'Essential DICOM tags missing'}) )
                    
                # Fill general info with other DICOM tags (if present)                
                self.PatientBirthDate      = self.ECG.PatientBirthDate      if set(['PatientBirthDate']).issubset(self.ECG.dir()) else ''
                self.PatientName           = self.ECG.PatientName           if set(['PatientName']).issubset(self.ECG.dir()) else ''
                self.PatientSex            = self.ECG.PatientSex            if set(['PatientSex']).issubset(self.ECG.dir()) else ''
                self.StudyDescription      = self.ECG.StudyDescription      if set(['StudyDescription']).issubset(self.ECG.dir()) else ''
                self.AcquisitionDateTime   = self.ECG.AcquisitionDateTime   if set(['AcquisitionDateTime']).issubset(self.ECG.dir()) else ''
                self.AcquisitionTimeZone   = self.ECG.TimezoneOffsetFromUTC if set(['TimezoneOffsetFromUTC']).issubset(self.ECG.dir()) else ''
                self.Manufacturer          = self.ECG.Manufacturer          if set(['Manufacturer']).issubset(self.ECG.dir()) else ''
                self.ManufacturerModelName = self.ECG.ManufacturerModelName if set(['ManufacturerModelName']).issubset(self.ECG.dir()) else ''
                self.SoftwareVersions      = self.ECG.SoftwareVersions      if set(['SoftwareVersions']).issubset(self.ECG.dir()) else ''
                self.DataExportedBy        = self.ECG.IssuerOfPatientID     if set(['IssuerOfPatientID']).issubset(self.ECG.dir()) else ''

                
                # Waveform information
                # Check existance of raw Waveform
                try:
                    self.Waveforms             = self.ECG.waveform_array(0).T
                except:
                    if (verbose==True): 
                        print('No ECG waveform present: inspect DICOM file')
                    return(pd.DataFrame({'filetype': [self.ECG.data_element('SOPClassUID').repval], 'error': 'No ECG waveform present'}) )
                
                # Add channel settings and lead information 
                settings                   = self.ECG.WaveformSequence[0].ChannelDefinitionSequence[0]
                self.ChannelSensitivity    = settings.ChannelSensitivity   if set(['ChannelSensitivity']).issubset(settings.dir()) else '' 
                self.ChannelBaseline       = settings.ChannelBaseline      if set(['ChannelBaseline']).issubset(settings.dir()) else '' 
                self.ChannelSampleSkew     = settings.ChannelSampleSkew    if set(['ChannelSampleSkew']).issubset(settings.dir()) else '' 
                self.FilterLowFrequency    = settings.FilterLowFrequency   if set(['FilterLowFrequency']).issubset(settings.dir()) else '' 
                self.FilterHighFrequency   = settings.FilterHighFrequency  if set(['FilterHighFrequency']).issubset(settings.dir()) else '' 
                self.NotchFilterFrequency  = settings.NotchFilterFrequency if set(['NotchFilterFrequency']).issubset(settings.dir()) else ''        
                self.lead_info_final       = self.lead_info(0)
                self.LeadVoltages          = self.make_leadvoltages(0)
                self.sf                    = self.ECG.WaveformSequence[0].SamplingFrequency

                # Check existance of MedianWaveform
                try: 
                    self.MedianWaveforms       = self.ECG.waveform_array(1).T
                    self.lead_info_final       = self.lead_info(1)
                    self.LeadVoltages2         = self.make_leadvoltages(1)
                    self.MedianWaveformPresent = 'Yes'
                except: 
                    self.MedianWaveformPresent = 'No'
                    self.MedianWaveforms       = ''
                    self.LeadVoltages2         = np.zeros((0,0))
                    if (verbose==True): 
                        print('No Median Waveform present')

                self.samplingfrequency     = self.resampling_500hz()

                # Create dictionary from the above 
                self.read_dict_final       = self.readable_dict()
            
            return self.read_dict_final

        except Exception as e:
            print(str(e))
            pass

    def readable_dict(self):
        """Make a readable dict"""
        read_dict                             = {}
        read_dict["SOPinstanceUID"]           = self.SOPinstanceUID
        read_dict["SERIESinstanceUID"]        = self.SERIESinstanceUID
        read_dict["STUDYinstanceUID"]         = self.STUDYinstanceUID
        read_dict["PatientID"]                = self.PatientID
        read_dict["PatientBirthDate"]         = self.PatientBirthDate
        read_dict["PatientName"]              = str(self.PatientName)
        read_dict["PatientSex"]               = self.PatientSex
        read_dict["StudyDate"]                = datetime.strptime(self.StudyDate, "%Y%m%d").strftime('%Y-%m-%d') #if your date is different format adapt
        read_dict["StudyTime"]                = self.StudyTime
        read_dict["StudyDescription"]         = self.StudyDescription
        read_dict["AcquisitionDateTime"]      = self.AcquisitionDateTime
        read_dict["AcquisitionTimeZone"]      = self.AcquisitionTimeZone
        #read_dict["DatapointsWaveform"]       = len(list(self.LeadVoltages.values())[0])
        #read_dict["DatapointsMedianWaveform"] = len(list(self.LeadVoltages2.values())[0])
        read_dict["AccessionNumber"]          = self.AccessionNumber
        read_dict["SamplingFrequency"]        = self.samplingfrequency
        read_dict["ChannelSensitivity"]       = self.ChannelSensitivity
        read_dict["ChannelBaseline"]          = self.ChannelBaseline
        read_dict["ChannelSampleSkew"]        = self.ChannelSampleSkew
        read_dict["FilterLowFrequency"]       = self.FilterLowFrequency
        read_dict["FilterHighFrequency"]      = self.FilterHighFrequency
        read_dict["NotchFilterFrequency"]     = self.NotchFilterFrequency
        read_dict["Manufacturer"]             = self.Manufacturer
        read_dict["ManufacturerModelName"]    = self.ManufacturerModelName
        read_dict["SoftwareVersions"]         = self.SoftwareVersions
        read_dict["DataExportedBy"]           = self.DataExportedBy
        read_dict["Waveforms"]                = self.LeadVoltages
        read_dict["MedianWaveforms"]          = self.LeadVoltages2
        return read_dict

    def make_leadvoltages(self,nr):
        """Extracts the voltages out of the DICOM"""
        num_leads = 0
        leads = {}

        for i, lead in enumerate(self.ECG.waveform_array(nr).T):
            num_leads += 1
            leads[self.lead_info_final[i]] = lead
        if num_leads == 8 and self.augmentLeads:
            leads['III'] = np.subtract(leads['II'], leads['I'])
            leads['aVR'] = np.add(leads['I'], leads['II']) * (-0.5)
            leads['aVL'] = np.subtract(leads['I'], 0.5 * leads['II'])
            leads['aVF'] = np.subtract(leads['II'], 0.5 * leads['I'])
        return leads

    def lead_info(self,nr):
        """returns the names of the channels from the DICOM"""
        leadnames = {}
        for ii, channel in enumerate(self.ECG.WaveformSequence[nr].ChannelDefinitionSequence):
            source = channel.ChannelSourceSequence[0].CodeMeaning
            units = "unitless"
            if "ChannelSensitivity" in channel:
                units = channel.ChannelSourceSequence[0].CodeMeaning
            leadnames[ii] = source.replace('Lead','').strip()
        return leadnames

    def resampling_500hz(self):
        """In case sf is 250, make 500"""
        if self.resample_500 is False:
            return int(self.sf)
        else:
            if int(self.sf) != 500:
                for i in self.LeadVoltages:
                    self.LeadVoltages[f"{i}"] = signal.resample(self.LeadVoltages[f"{i}"], 5000)
                    self.oversampled = "Yes"
                if self.MedianWaveformPresent == 'Yes':
                    for i in self.LeadVoltages:
                        self.LeadVoltages2[f"{i}"] = signal.resample(self.LeadVoltages2[f"{i}"], 600)
                self.sf = 500
                return self.sf
            else:
                return self.sf
        return self

In [ ]:
#Initialize
ecgreader = ECGDICOMReader()

In [ ]:
# Define path
path_to_dicom = "C:/Users/Stephan.vanderZwaard/Documents/dicom/ECG-DICOM/"

# Obtain all DICOM files
files = os.listdir(path_to_dicom)

In [ ]:
# Get unique values from files list
len(files)
len(list(set(files)))

In [ ]:
# Read ECG from DICOM


# Preallocate dataframes
general_info   = pd.DataFrame()
median_waves   = pd.DataFrame()
original_waves = pd.DataFrame()

# Set-up progressbar
i_start = 21000
i_end   = 21433
pbar    = progressbar.ProgressBar(maxval = len(files[i_start:i_end])-1).start()

for i in range(i_start,i_end) : 
    
    try:
    dicom = ecgreader(path_to_dicom + files[i])

    # Generate Table 1: median waveform
    mw = pd.DataFrame(dicom['MedianWaveforms'])
    mw = mw.add_prefix('lead_')
    mw["id"] = mw.index
    mw = pd.wide_to_long(mw, stubnames ='lead_', i="id", j="lead",suffix = '\w+').sort_index(level=0)
    mw["SOPinstanceUID"] = dicom["SOPinstanceUID"]
    mw["waveform"] = "median_beat"
    mw = mw.reset_index()
    mw = mw[["SOPinstanceUID", "waveform", "lead","id","lead_"]]
    mw = mw.rename(columns = {'lead_':'voltage'})
    

    # Generate Table 2: waveform rhythm
    w = pd.DataFrame(dicom['Waveforms'])
    w = w.add_prefix('lead_')
    w["id"] = w.index
    w = pd.wide_to_long(w, stubnames ='lead_', i="id", j="lead",suffix = '\w+').sort_index(level=0)
    w["SOPinstanceUID"] = dicom["SOPinstanceUID"]
    w["waveform"] = "rhythm"
    w = w.reset_index()
    w = w[["SOPinstanceUID", "waveform","lead", "id","lead_"]]
    w = w.rename(columns = {'lead_':'voltage'})


    # Generate Table 3: general info
    wave = dicom.pop('Waveforms')
    mbeat= dicom.pop('MedianWaveforms')
    info = pd.DataFrame.from_dict(dicom, orient = 'index').transpose()

    
    # Combine data with other records
    general_info    = pd.concat([general_info,info], axis=0)
    median_waves    = pd.concat([median_waves,mw], axis=0)
    original_waves  = pd.concat([original_waves,w], axis=0)
    
    # Update progressbar
    pbar.update(i-i_start)


In [ ]:
# Save to CSV-files
batch = '21000-21433'

# Path
path = 'C:/Users/Stephan.vanderZwaard/Documents/dicom/ECG-DICOM-results/'

# Save separate CSV-files for each batch
median_waves.to_csv(path+'DICOM_ECG_WAVEFORM_MEDIANBEAT_'+batch+'.csv', index=False)
original_waves.to_csv(path+'DICOM_ECG_WAVEFORM_RHYTHM_'+batch+'.csv', index=False)
general_info.to_csv(path+'DICOM_ECG_GENERALINFO_'+batch+'.csv', index=False)


In [ ]:
general_info

In [ ]:
# Print results from read dicom file
dicom

In [ ]:
#Plot median waveform from dicom file
plt.plot(mbeat["V5"])

In [ ]:
#Plot waveform from dicom file
plt.plot(wave["V5"])

In [ ]:
# Define path
path_to_dicom = "C:/Users/Stephan.vanderZwaard/Documents/dicom/ECG-DICOM/1.3.6.1.4.1.40744.65.103877278949773242685639440347979340500.dcm"

# Read ECG from DICOM
#dicom = dcmread(path_to_dicom)
dicom = ecgreader(path_to_dicom)

In [ ]:
dicom                

In [ ]:
# Define path
path_to_dicom = "C:/Users/Stephan.vanderZwaard/Documents/dicom/1.3.6.1.4.1.40744.65.179071579855649261499076763286137160274.dcm"

# Read ECG from DICOM
dicom = ecgreader(path_to_dicom)
#dicom = dcmread(path_to_dicom)

# Bevestigd door
# Ventricular HR
# Atrial HR
# QRS duration
# QT interval
# QTc interval
# R-axis
# T-axis


In [ ]:
dicom

In [ ]:
# Generate Table 1: median waveform
mw = pd.DataFrame(dicom['MedianWaveforms'])
mw = mw.add_prefix('lead_')
mw["id"] = mw.index
mw = pd.wide_to_long(mw, stubnames ='lead_', i="id", j="lead",suffix = '\w+').sort_index(level=0)
mw["SOPinstanceUID"] = dicom["SOPinstanceUID"]
mw["waveform"] = "median_beat"
mw = mw.reset_index()
mw = mw[["SOPinstanceUID", "waveform", "lead","id","lead_"]]
mw = mw.rename(columns = {'lead_':'voltage'})
#mw.to_csv('DICOM_ECG_WAVEFORM_MEDIANBEAT.csv', index=False)


# Generate Table 2: waveform rhythm
w = pd.DataFrame(dicom['Waveforms'])
w = w.add_prefix('lead_')
w["id"] = w.index
w = pd.wide_to_long(w, stubnames ='lead_', i="id", j="lead",suffix = '\w+').sort_index(level=0)
w["SOPinstanceUID"] = dicom["SOPinstanceUID"]
w["waveform"] = "rhythm"
w = w.reset_index()
w = w[["SOPinstanceUID", "waveform","lead", "id","lead_"]]
w = w.rename(columns = {'lead_':'voltage'})
#w.to_csv('DICOM_ECG_WAVEFORM_RHYTHM.csv', index=False)


# Generate Table 3: general info
wave = dicom.pop('Waveforms')
mbeat= dicom.pop('MedianWaveforms')
info = pd.DataFrame.from_dict(dicom, orient = 'index').transpose()
#info.to_csv('DICOM_ECG_GENERALINFO.csv', index=False)


In [ ]:
dicom = dcmread(path_to_dicom)
dicom.TimezoneOffsetFromUTC

In [ ]:
dicom[0x00120064].name
dicom[0x0040b020][0][0x00700006].value
dicom[0x004008ea][0][0x00700006].value

## TEST ECG FROM SET MARYAM (VUMC LOCATION)

In [ ]:
# Define path
path_to_dicom = "C:/Users/Stephan.vanderZwaard/Documents/dicom/TEST_ECGs/"

# Obtain all DICOM files
ECG_files = list() 
for path, subdirs, files in os.walk(path_to_dicom):
    for filename in files:
        ECG_files.append(path+'/'+filename)

In [ ]:
dicom = ecgreader(ECG_files[10])
#dicom = dcmread(ECG_files[11])

In [ ]:
dicom

In [ ]:
dicom = dcmread(ECG_files[17])
dicom.dir()
#print(dicom[0x00020002])

In [ ]:
len(ECG_files)

In [ ]:
# Read ECG from DICOM


# Preallocate dataframes
general_info   = pd.DataFrame()
median_waves   = pd.DataFrame()
original_waves = pd.DataFrame()
error_dicom    = pd.DataFrame()

# Set-up progressbar
i_start = 0
i_end   = 23
pbar    = progressbar.ProgressBar(maxval = len(ECG_files[i_start:i_end])-1).start()

for i in range(i_start,i_end) : 
    
    try:
        dicom = ecgreader(ECG_files[i], verbose=False)

        # Generate Table 1: median waveform
        mw = pd.DataFrame(dicom['MedianWaveforms'])
        mw = mw.add_prefix('lead_')
        mw["id"] = mw.index
        mw = pd.wide_to_long(mw, stubnames ='lead_', i="id", j="lead",suffix = '\w+').sort_index(level=0)
        mw["SOPinstanceUID"] = dicom["SOPinstanceUID"]
        mw["waveform"] = "median_beat"
        mw = mw.reset_index()
        mw = mw[["SOPinstanceUID", "waveform", "lead","id","lead_"]]
        mw = mw.rename(columns = {'lead_':'voltage'})


        # Generate Table 2: waveform rhythm
        w = pd.DataFrame(dicom['Waveforms'])
        w = w.add_prefix('lead_')
        w["id"] = w.index
        w = pd.wide_to_long(w, stubnames ='lead_', i="id", j="lead",suffix = '\w+').sort_index(level=0)
        w["SOPinstanceUID"] = dicom["SOPinstanceUID"]
        w["waveform"] = "rhythm"
        w = w.reset_index()
        w = w[["SOPinstanceUID", "waveform","lead", "id","lead_"]]
        w = w.rename(columns = {'lead_':'voltage'})


        # Generate Table 3: general info
        wave = dicom.pop('Waveforms')
        mbeat= dicom.pop('MedianWaveforms')
        info = pd.DataFrame.from_dict(dicom, orient = 'index').transpose()


        # Combine data with other records
        general_info    = pd.concat([general_info,info], axis=0)
        median_waves    = pd.concat([median_waves,mw], axis=0)
        original_waves  = pd.concat([original_waves,w], axis=0)
    
    except:
        dicom['file_no'] = i
        dicom['filename'] = ECG_files[i].replace('C:/Users/Stephan.vanderZwaard/Documents/dicom/','')
        error_dicom     = pd.concat([error_dicom,dicom], axis=0)
        
    # Update progressbar
    pbar.update(i-i_start)


In [ ]:
median_waves.loc[median_waves['SOPinstanceUID'] == '1.3.6.1.4.1.40744.65.214950033209762893283513913898962270513']

In [ ]:
error_dicom

In [ ]:
error_dicom.to_csv('DICOM_error.csv', index=False)
general_info.to_csv('DICOM_GENERAL_INFO.csv', index=False)


In [ ]:
ECG_files

In [ ]:
plt.plot(dicom["Waveforms"]["V2"])

In [ ]:
dicom

In [ ]:
i